In [43]:
import numpy as np
import matplotlib.pyplot as plt

In [44]:
def u0(x):
    return x * (x + 1)

def u2(t):
    return 2 * np.cos(np.pi * t)

def u(t, x):
    return u0(x) * np.cos(np.pi * t)

def F(t, x):
    return -np.cos(np.pi * t) * (np.pi**2 * x * (x + 1) + 2)

In [45]:
x_limits = [0, 1]
t_limits = [0, 10] # ?
nx = 100 # x steps
nt = 10000 # t steps

u1 = 0

x = np.linspace(*x_limits, nx)
dx = x[1] - x[0]
t = np.linspace(*t_limits, nt)
dt = t[1] - t[0]


In [46]:
dx, dt, dt**2/dx**2

(np.float64(0.010101010101010102),
 np.float64(0.001000100010001),
 np.float64(0.009802960494069209))

In [47]:
# y = u(t, x)

# plt.plot(x, y, "b-", x, y, "r.")
# plt.xlabel("x")
# plt.ylabel("y")
# plt.show()